In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from whatlies import Embedding, EmbeddingSet
import spacy 

## Making Plots ... More Cool 

The `Embedding` object merely has support for matplotlib, but the `EmbeddingSet` has support for Altair too! You can plot this interactively by just passing the names of the tokens you'd like to see.

In [3]:
nlp = spacy.load("en_core_web_md")
words = ["prince", "princess", "nurse", "doctor", "banker", "man", "woman", 
         "cousin", "neice", "king", "queen", "dude", "guy", "gal", "fire", 
         "dog", "cat", "mouse", "red", "bluee", "green", "yellow", "water", 
         "person", "family", "brother", "sister"]
emb = EmbeddingSet({t.text: Embedding(t.text, t.vector) for t in nlp.pipe(words)})

In [4]:
orig_chart = emb.plot_interactive('man', 'woman')
orig_chart

alt.LayerChart(...)

In [5]:
new_ts = emb | (emb['king'] - emb['queen'])
new_chart = new_ts.plot_interactive('man', 'woman')

Note that altair has a convenient syntax for plotting two charts next to eachother. This is really cool when you want to compare. Feel free to zoom in and play as well!

In [6]:
orig_chart | new_chart

alt.HConcatChart(...)

The charts that we output here are from the Altair library. This means that you can, among other things, customise the size if you prefer.

In [7]:
s = 250
(orig_chart.properties(width=s, height=s) | new_chart.properties(width=s, height=s))

alt.HConcatChart(...)

## Transformations

There's something extra too. Sofar we're been mapping vectors unto other ones in order to plot them. But theoretically we could go a step further.

In [20]:
from whatlies.transformers import Pca, Umap

orig_chart = emb.plot_interactive('man', 'woman')
pca_emb = emb.transform(Pca(2))
umap_emb = emb.transform(Umap(2))

pca_emb.plot_interactive('pca_0', 'pca_1') | umap_emb.plot_interactive('umap_0', 'umap_1')

alt.HConcatChart(...)

Note that we can increase the number of components and still only plot a few. 

In [21]:
pca_emb = emb.transform(Pca(3))

pca_emb.plot_interactive('pca_0', 'pca_1').properties(width=s, height=s) | pca_emb.plot_interactive('pca_2', 'pca_1').properties(width=s, height=s)

alt.HConcatChart(...)

But why go with only two plots when you can have an entire matrix? 

In [22]:
pca_emb.plot_interactive_matrix('pca_0', 'pca_1', 'pca_2')

alt.RepeatChart(...)

What is particularily interesting here is the pca axes. They seem to encode information and can we attempt an understanding by glancing at it.

But the overlap makes it hard to read. So let's apply one more transformation here.

In [25]:
from whatlies.transformers import Noise 

(emb
 .transform(Pca(3))
 .transform(Noise(2))
 .plot_interactive_matrix('king', 'queen', 'man', 'woman', annot=True, width=200, height=200))

alt.RepeatChart(...)

Note that we also offer the ability to add a few random embeddings. This can be a useful sanity check.

In [45]:
from whatlies.transformers import AddRandom

(emb
 .transform(AddRandom(n=10, sigma=0.1))
 .transform(Pca(2))
 .plot_interactive_matrix('pca_0', 'pca_1', annot=True, width=200, height=200))

alt.RepeatChart(...)